# **`A Naïve Bayes Classifier for Text Classification`**
# **Candidate: *Martina Toffoli VR446059***

The assignment consists in the development, through the use of NLTK, of a Naïve Bayes Classifier able to detect a single class in one of the corpora available as attachments to the chosen package, by distinguishing ENGLISH against NON-ENGLISH.

This first block is related to the main libraries needed to compute the classification task. Most of the produced code is based on NLTK which is the Natural Language Toolkit used to achieve our goal.

In [2]:
import re
from os import listdir
from os.path import isdir, join
import nltk
nltk.download('europarl_raw')
nltk.download('punkt')
from nltk.corpus.europarl_raw import english, french, danish, dutch, finnish, german, greek, portuguese, italian, spanish, swedish
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import heapq
import numpy as np
import csv
import random
import string
from nltk.stem import PorterStemmer
from nltk.metrics.confusionmatrix import ConfusionMatrix
from nltk.metrics import recall, precision, f_measure
from collections import defaultdict

[nltk_data] Downloading package europarl_raw to /root/nltk_data...
[nltk_data]   Package europarl_raw is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Some function have been subsequently created respectively to preprocess the text, compute the words occurrences and to extract the relevant features from the data:

*   text_converter(text)
*   freqWords(dataset, minFreq)
*   find_features(sentence, freq_words)


# Function: **text_converter(text)**
During the analysis of a particular text, the model better understand the content of the input if some preprocessing has been done. Usually, lowerizing the input text deleting the puntcuation are classical operations to improve the performance of a machine learning model.
We demonstrate that the capability of language text classification can be learned feeding the model with inputs that are sentences and not necessarily an entire text. For this reason we tokenize the input using a sentence tokenizer and we use each sentence as if it were a different sample text.
This consideration, enable us to train the model with a limit amount of data and obtaining high performance. Therefore, we read a long text for each language, we apply a sentence tokenization on it obtaining a list of sentences used for training the model.

**[Concept of Tokenization]:** It is the process by which a large quantity of text is divided into smaller parts called tokens.

**[Input]:** text

**[Output]:** list of text

In [3]:
def text_converter(testo):
    # STEP 1: We will first preprocess the data, in order to:
    dataset = nltk.sent_tokenize(testo)
    for i in range(len(dataset)):
        # remove numbers
        sentence_ith = re.sub(r'\d+', '', dataset[i])
        # remove punctuations and convert characters to lower case
        dataset[i] = "".join([char.lower() for char in sentence_ith if char not in string.punctuation])
    return dataset

# Function: **freqWords(dataset, minFreq)**
This procedure acts as a support function which helps us to create the Bag of Words (BOW) that is a method to extract features from text documents. In simple terms, it returns a collection of tokens which represents the occurrences of words in the text disregarding the order in which they appear. This is useful because when we analyze the content of the text, we inherently associate a weigth to each word. For this reason we decide to make the model concentrate on the most redundant words in the text. 

**[Input]:** text, number of different ***n*** words to consider as most relevant

**[Output]:** list of most ***n*** relevant words

In [4]:
def freqWords(dataset, minFreq):
    # STEP 2: Obtaining most frequent words in our text.
    # Creating the Bag of Words model
    word2count = {}
    ps = PorterStemmer()
    for sentence in dataset:
        words = nltk.word_tokenize(sentence)
        for word in words:
            word = ps.stem(word)
            if word not in word2count.keys():
                word2count[word] = 1
            else:
                word2count[word] += 1
    # estrapolo le 200 parole più frequenti nel testo
    freq_words = heapq.nlargest(minFreq, word2count, key = word2count.get)
    #print(freq_words)
    return freq_words

# Function: **find_features(sentence, freq_words)**
The purpose of this function consists of extracting/finding the most relevant features in the text as explained above. In particular, providing in input the text and the list of the most ***n*** frequent words, we maintain only the words that appears in that list.

**[Input]:** text, list of most frequents words

**[Output]:** dictionary {word: word}

In [5]:
def find_features(sentence, freq_words):
    features = {}
    words = nltk.word_tokenize(sentence)
    for word in words:
        if word in freq_words:
            features[word] = word
    
    return features

The classifier was implemented starting from the ***europarl_raw*** corpus present in the NLTK package.

It contains documents written in diffent languages: French, Italian, Spanish, Portuguese, German, English, Dutch, Danish, Swedish, Finnish and Greek.
We decide to use it because its size, number of languages and number of documents are appropriate for this specific task.

All the texts of each language have been modified through the *text_converter(text)* function in order to divide it into several different texts; subsequently the most frequent words within the text were detected through the *freqWords(sentences_list,minFreq)* function; finally we extract the main features on which the classifier will have to operate applying the *find_features(sentence,frequents_words)* function. 
Therefore, for each sample we extract the relative features in the form of a dictionary as explained above. In addition, we insert it into a tuple paired with its relative label which indicates whether this sentence is in English or not (English or Not-English).

Example of features:

                feature = ({'the' : 'the', 
                            'pen' : 'pen',
                             'is' : 'is', 
                             'on' : 'on',
                            'the' : 'the',
                          'table' : 'table',}, 'English')


                              

In [6]:
max_elem = 100000

dataset = []
text = ' '.join(english.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'English') for sentence in sentences_list]
dataset.extend(features[:max_elem])

text = ' '.join(dutch.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(finnish.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(german.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(greek.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(portuguese.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(italian.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(spanish.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(swedish.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(french.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

text = ' '.join(danish.words())
sentences_list = text_converter(text)
frequents_words = freqWords(sentences_list,200)
features = [(find_features(sentence,frequents_words), 'Not-English') for sentence in sentences_list]
dataset.extend(features[:max_elem//10])

The final dataset is a set of balanced samples containing text in English and not in equal numbers; this allows the classifier to interpret more accurately whether a text is in English or not.

A *random.shuffle(dataset)* operation is performed before training the model to mix all the samples so as not to have a list of ordered text by language, enabling the classifier to learn better from a non homogenous dataset.


In [7]:
random.shuffle(dataset)

For this assignment we decided to use the Naïve Bayes Classifier. In order to find the probability for a label, this algorithm first uses the Bayes rule to express P(label|features) in terms of P(label) and P(features|label):
                     
*P(label|features) = ( P(label) * P(features|label) ) / P(features)*

Moreover, it makes use of the 'naïve' assumption that all features are
independent as shown in the following expression:

*P(label|features) = ( P(label) * P(f1|label) * ... * P(fn|label) ) / P(features)*

In addition, rather than computing P(features) explicitly, the algorithm just
calculates the numerator for each label, and normalizes them so they
sum to one:

*P(label|features) = ( P(label) * P(f1|label) * ... * P(fn|label) ) / ( SUM(l)( P(l) * P(f1|l) * ... * P(fn|l) )*

To train the model, it is necessary to split the dataset into a training set and a test set: 
The training set is used to allow the algorithm to distinguish English texts with respect to other languages while the test set is useful to evaluate how well the algorithm works, namely the performance of our machine learning model.

In this assignment, we use 10% of the entire dataset during the evaluation (test set) and the the remaining part for the training phase (training set).

In [8]:
train_set, test_set = dataset[max_elem//10:], dataset[:max_elem//10]
classifier = nltk.NaiveBayesClassifier.train(train_set)

# **Performance indicators**
For this assignment it was decided to use a series of indicators that allow us to interpret the performance of the classifier, including:


*   **Confusion Matrix**: It is crucial to verify the errors predictions and to understand where the model makes the wrong choice. 
*   **Accuracy**: Given a list of reference values and a corresponding list of test values, return the fraction of corresponding values that are equal.
*   **Precision**: Given a set of reference values and a set of test values, return the fraction of test values that appear in the reference set.
*   **Recall**: Given a set of reference values and a set of test values, return the fraction of reference values that appear in the test set.
*   **F-Measure**:  Given a set of reference values and a set of test values, return the f-measure of the test values, when compared against the reference values.  The f-measure is the harmonic mean of the precision and recall.


In [9]:
groundtruth = [ sample[1] for sample in test_set]
predictions = []
for sample, _ in test_set:
    prediction = classifier.classify(sample)
    predictions.append(prediction)

print("Confusion Matrix:\n")
print(ConfusionMatrix(groundtruth, predictions))

groundtruth, predictions = defaultdict(set), defaultdict(set)

for i, (features, label) in enumerate(test_set):
    predictions[classifier.classify(features)].add(i)
    groundtruth[label].add(i) 
print(" Accuracy:", nltk.classify.accuracy(classifier, test_set))
print("Precision:", precision(groundtruth[label], predictions[label]))
print("   Recall:", recall(groundtruth[label], predictions[label]))
print("F-Measure:", f_measure(groundtruth[label], predictions[label]))

Confusion Matrix:

            |         N |
            |         o |
            |         t |
            |         - |
            |    E    E |
            |    n    n |
            |    g    g |
            |    l    l |
            |    i    i |
            |    s    s |
            |    h    h |
------------+-----------+
    English |<1624>   4 |
Not-English |    2<8370>|
------------+-----------+
(row = reference; col = test)

 Accuracy: 0.9994
Precision: 0.9995223310246
   Recall: 0.9997611084567606
F-Measure: 0.9996417054819061


# **Conclusion**
We demostrated to obtain optimal results.
Considerations: 
*   This is due to a large amount of data, namely the quantity of english features is equal of the quantity of the not english features. 
*   The use of several and different metrics is a good indicator for the perfomance of the model. Therefore we use not only precision but also recall, accuracy and f-measure.
*   As shown in the confusion matrix, just only very few texts are wrong classify.
*   We almost reach the optimal point.

# **Future Consideration**
*   Different tokenizer could be tested.
*   The function used to extract the features can be enhance using not just the most relevant words but also some chunks of the text, avoiding stopwords and so on...
*   A different classifier could be used in order to analyze the strenghts and weaknesses of different models.
*   It would be interesing to analyze what happens to the model performance to vary the amount of data used to feed the model during the training phase.

